In [3]:
#######This code is for automatically calculating and exporting two of the features within 
##the machine learning model to an excel sheet ('friday short volume' and 'friday short volume-5 day average')

## It functions by taking in a list of stock tickers at the stock variable
##it then gets the short volume data from the fintel webpage 
##by digesting the html code and pulling out the relevant information
##for a respective stock
##Fintel has throttling on its website (will block your IP if you make too many requests
##during a given time frame) To deal with this i am making my requests through the 
##application Tor, and using it to change my IP address every request
##In order to use this you need to download the Tor application: https://www.torproject.org/download/
#and then download the torrequest python library and configure one of the files within the 
##torrequest package by adding the tor application executable file path to a line of 
#python code, process described in detail here: https://stackoverflow.com/questions/24801924/tor-with-python-stem-basic-tor-not-in-path
#(the second answer is the one that describes what i did)

#Inputs you need to make:
#1. specify export file path location
#2. specify your stock ticker list
#3. specify dates in your week
#4. specify friday
##This hasnt been completely trouble shot, it will throw a couple of different errors 
## that mostly deal with processing the html code. When that happens the code will export
##'NaN' for the feature export values associated with that ticker
## One error that has been successfully dealt with is 'days missing'. For whatever reason
##stocks sometimes won't have a day of short volume data, i deal with this by counting how many 
##times that this happens and exporting it to the sheet. 
#any stock with more than one day of short data missing and or friday short data missing gets
#doesn't get included in the machine learning model.  
import requests
import re 
import string
import time 
import numpy as np 
import pandas as pd 
import xlsxwriter
from torrequest import TorRequest
import requests
from torpy import TorClient
import os 
import torpy
from torpy.http.requests import TorRequests
from multiprocessing.pool import ThreadPool
from torpy.http.requests import tor_requests_session
from stem.control import Controller
import stem 


torprocess = stem.process.launch_tor(tor_cmd = '/Applications/Tor Browser.app/Contents/Resources/TorBrowser/Tor/tor')
cookies = dict(BCPermissionLevel='PERSONAL')
proxy = {'http':  'socks5://127.0.0.1:9050',
           'https': 'socks5://127.0.0.1:9050'}
   # return session
#################################specify export file location############################################
workbook=xlsxwriter.Workbook('/Users/jpjansen/Documents/Shorts_3_28_v1.xlsx')
##############################################################################################
worksheet = workbook.add_worksheet()
worksheet.write(0,0,'Stock Symbol')
worksheet.write(0,1,'Friday Short Ratio')
worksheet.write(0,2,'Friday-Week Average')
worksheet.write(0,3,'Week Average')
worksheet.write(0,4,'Days Missing')
######''#########stock list####################################################################
stock = ['FUBO', 'TME', 'WAFU', 'NEXT', 'XSPA', 'CIDM', 'EEIQ', 'NAKD', 'CLEU', 'GBR', 'HSDT', 'BB','DS', 'KDMN', 'OXY', 'GNPK', 'HOL', 'NPA', 'NSH', 'SFTW', 'SRAC', 'WIMI', 'LGHL', 'NAT', 'SIEN', 'HGEN', 'OXBR', 'DFFN', 'EVFM', 'HEPA', 'CLBS', 'HUGE', 'ENVB', 'DSS', 'EBON', 'CSCW', 'RAIL', 'ENSV', 'HUSA', 'AYRO', 'VISL', 'BRQS', 'BYFC', 'BOWX', 'DLPN', 'HOFV', 'JFIN', 'LMPX', 'UAVS', 'AMC', 'LIFE', 'ATNF', 'AACG', 'ORN', 'PLUG', 'CASI','ZDGE', 'FLNT', 'CURI', 'MX', 'TH', 'GHSI', 'KNDI', 'IZEA', 'MTNB', 'OEG', 'PRQR', 'TKAT', 'RCON', 'AJAX']
url = "https://fintel.io/ss/us/"
allow = string.ascii_lowercase[1]+string.digits+"="+"."+"-"
friday_shorts = []
week_average = []
friday_subtractmean = [] 
counter = 0 

for k in range(len(stock)):
    loopURL = url+stock[k]
    with TorRequest(proxy_port=9050, ctrl_port=9051, password = 'password') as tr:
        res = tr.get(loopURL, proxies = proxy, headers={"User-Agent": "Mozilla/5.0"}, cookies=cookies)
        ipaddress= tr.get('http://ipecho.net/plain')
        print ("New Ip Address",ipaddress.text)
        tr.reset_identity()
        tr.close()
    print(stock[k])

    a = res.content
    b = a.decode('UTF-8')
    c = b.replace('"', '')
    c = c.replace("'", '')
    c = c.replace(",", '')
    c = c.replace(";",'')
    c = c.replace(">",'')
    c = c.replace("<",'')
    c = c.replace("/",'')
    c = c.replace("/",'')
    c = c.replace('','')
    c = c.replace("\r","")
    c = c.replace("\n","")
   
    try:
        d = c.split("Short Volume Ratio adivdiv class=col-xs-12 col-sm-12 col-md-12 col-lg-6table id=topic-table-body")
        e = d[1]
        f = e.split('tdtrtbodytabledivdivdivdiv')
    except IndexError:
        friday = 'NaN'
        week_average = 'NaN'
        friday_subtractmean = 'NaN'
        print(stock[k],'INDEX ERROR')
        
        worksheet.write(counter+1,0,stock[k])
        worksheet.write(counter+1,1,friday)
        worksheet.write(counter+1,2,friday_subtractmean)
        worksheet.write(counter+1,3,week_average)
        counter+=1
        time.sleep(1)
        continue 
  
    asdf = re.sub('[^%s]'% allow, '', f[0])
    testing = asdf.split('=')
    data = []
    for i in range(len(testing)):
        z = testing[i].split('b')
        for q in range(len(z)):
            if z[q]!='':
                if z[q]!='-':
                    data.append(z[q])
    #print(data)
###############################specify dates in week############################################################                
    previous_week_dates = ['2021-03-22','2021-03-23','2021-03-24','2021-03-25','2021-03-26'] 
############################################################################################
    try:
        shortvolume = []
        totalvolume=[]
        shortratio = [] 
        for i in range(len(previous_week_dates)):
            for z in range(len(data)):
                if data[z]== previous_week_dates[i]:
                    #print('MATCH')
                    #print('SV', data[z+1])
                    #print('TV', data[z+2])
                    #print('SR', data[z+3])
                    shortvolume.append(data[z+1])
                    totalvolume.append(data[z+2])
                    shortratio.append(data[z+3])
                    break
        SVfloat = []
        TVfloat = []
        SRfloat = []
        for i in range(len(shortratio)):
            SVfloat.append(np.abs(float(shortvolume[i])))
            TVfloat.append(np.abs(float(totalvolume[i])))
            SRfloat.append(np.abs(float(shortratio[i])))
        #print(SVfloat,TVfloat,SRfloat)
        #dates = np.transpose(previous_week_dates)
        #final_data = np.column_stack((dates, SVfloat, TVfloat, SRfloat))
        #print(final_data)
        #df = pd.DataFrame(data=final_data[:,1:], index=[dates], columns=["short volume", "total volume", "short ratio"])
        try:
            dates = np.transpose(previous_week_dates)
            final_data = np.column_stack((dates, SVfloat, TVfloat, SRfloat))
            #print(final_data)
            df = pd.DataFrame(data=final_data[:,1:], index=[dates], columns=["short volume", "total volume", "short ratio"])
####################################specify friday date #############################################################
            friday = float(df.loc['2021-03-26','short ratio'].values[0])
            print('Friday = ', friday)
#################################################################################################
            friday_shorts.append(friday)
        except KeyError:
            friday = 'NaN'
            week_average = 'NaN'
            friday_subtractmean = 'NaN'
            print(stock[k],'INDEX ERROR')

            worksheet.write(counter+1,0,stock[k])
            worksheet.write(counter+1,1,friday)
            worksheet.write(counter+1,2,friday_subtractmean)
            worksheet.write(counter+1,3,week_average)
            counter+=1
            time.sleep(1)
            continue 
   
        short_ratios = [] 
        days_missing = 0 
        for i in range(len(previous_week_dates)):
            try:
                short_ratios.append(float(df.loc[previous_week_dates[i],'short ratio'].values[0]))
            except KeyError:
                days_missing+=1
        print('short ratios',short_ratios)
        print('Fridays',friday)
        week_average = np.sum(short_ratios)/len(short_ratios)
        print('week average', week_average)
        friday_subtractmean = friday-week_average
        print('friday minus mean',friday_subtractmean)
    
    except ValueError:
        print("Value Error", stock[k],shortvolume[i])
        week_average = 'NaN'
        friday_subtractmean = 'NaN'
    try:
        print(df)
    except NameError:
        friday = 'NaN'
        days_missing = 'NaN'
        print('Name Error')
    worksheet.write(counter+1,0,stock[k])
    worksheet.write(counter+1,1,friday)
    #print('friday_subtractmean', friday_subtractmean)
    worksheet.write(counter+1,2,friday_subtractmean)
    worksheet.write(counter+1,3,week_average)
    worksheet.write(counter+1,4,days_missing)
    worksheet.write(counter+1,5,ipaddress.text)

    counter+=1
    time.sleep(1)
workbook.close()


New Ip Address 185.220.102.8
FUBO
Friday =  13.34
short ratios [13.54, 13.58, 23.1, 17.06, 13.34]
Fridays 13.34
week average 16.124000000000002
friday minus mean -2.7840000000000025
           short volume total volume short ratio
2021-03-22     629097.0         4.64       13.54
2021-03-23    1011261.0         7.45       13.58
2021-03-24    1622872.0         7.02        23.1
2021-03-25    1641913.0         9.62       17.06
2021-03-26    4226142.0        31.68       13.34
New Ip Address 109.70.100.56
TME
Friday =  6.47
short ratios [6.57, 10.94, 12.52, 11.54, 6.47]
Fridays 6.47
week average 9.607999999999999
friday minus mean -3.137999999999999
           short volume total volume short ratio
2021-03-22    1099515.0        16.74        6.57
2021-03-23    2530653.0        23.13       10.94
2021-03-24    5458778.0        43.61       12.52
2021-03-25    6579478.0         57.0       11.54
2021-03-26   30214964.0       466.99        6.47
New Ip Address 185.220.101.202
WAFU
Friday =  33.96
sh

NSH
Friday =  5.63
short ratios [13.65, 19.14, 20.56, 9.45, 5.63]
Fridays 5.63
week average 13.685999999999998
friday minus mean -8.055999999999997
           short volume total volume short ratio
2021-03-22     105703.0         0.77       13.65
2021-03-23     153556.0          0.8       19.14
2021-03-24     199530.0         0.97       20.56
2021-03-25     122576.0          1.3        9.45
2021-03-26      80353.0         1.43        5.63
New Ip Address 199.249.230.78
SFTW
Friday =  26.25
short ratios [18.34, 19.71, 23.01, 22.91, 26.25]
Fridays 26.25
week average 22.044
friday minus mean 4.2059999999999995
           short volume total volume short ratio
2021-03-22     467380.0         2.55       18.34
2021-03-23     429095.0         2.18       19.71
2021-03-24     563532.0         2.45       23.01
2021-03-25     560499.0         2.45       22.91
2021-03-26     459731.0         1.75       26.25
New Ip Address 185.220.101.22
SRAC
Friday =  16.69
short ratios [17.98, 15.94, 15.53, 17.83, 

New Ip Address 23.129.64.202
ENSV
Friday =  17.86
short ratios [26.67, 17.79, 34.45, 17.41, 17.86]
Fridays 17.86
week average 22.836
friday minus mean -4.975999999999999
           short volume total volume short ratio
2021-03-22      78067.0         0.29       26.67
2021-03-23      77199.0         0.43       17.79
2021-03-24     170122.0         0.49       34.45
2021-03-25      65699.0         0.38       17.41
2021-03-26      39971.0         0.22       17.86
New Ip Address 185.220.100.254
HUSA
Friday =  19.62
short ratios [16.06, 21.76, 24.59, 17.52, 19.62]
Fridays 19.62
week average 19.91
friday minus mean -0.28999999999999915
           short volume total volume short ratio
2021-03-22     189025.0         1.18       16.06
2021-03-23     151358.0          0.7       21.76
2021-03-24     435013.0         1.77       24.59
2021-03-25     106107.0         0.61       17.52
2021-03-26      92426.0         0.47       19.62
New Ip Address 46.38.235.14
AYRO
Friday =  12.97
short ratios [12.27,

New Ip Address 193.218.118.231
CASI
Friday =  29.03
short ratios [20.74, 20.76, 16.62, 14.52, 29.03]
Fridays 29.03
week average 20.334
friday minus mean 8.696000000000002
           short volume total volume short ratio
2021-03-22      32873.0         0.16       20.74
2021-03-23      71308.0         0.34       20.76
2021-03-24     750146.0         4.51       16.62
2021-03-25     176011.0         1.21       14.52
2021-03-26    1148972.0         3.96       29.03
New Ip Address 185.220.100.251
ZDGE
Friday =  13.68
short ratios [19.06, 11.25, 10.49, 12.52, 13.68]
Fridays 13.68
week average 13.4
friday minus mean 0.27999999999999936
           short volume total volume short ratio
2021-03-22     212209.0         1.11       19.06
2021-03-23      53762.0         0.48       11.25
2021-03-24      47181.0         0.45       10.49
2021-03-25      90665.0         0.72       12.52
2021-03-26      77788.0         0.57       13.68
New Ip Address 185.220.100.247
FLNT
Friday =  8.16
short ratios [13.66

In [4]:
print(os.getcwd())

/Users/jpjansen
